<a href="https://colab.research.google.com/github/ecaldwe1/DSCI502_DataMiningAtScale/blob/main/Week7_FlajoletMartinAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stream Processing - Part 3

## Counting Unique Elements

We suppose now that we are interested in determining the number of unique elements that have been seen in the stream since some time in the past.

A naive approach would be to store all unique elements in main memory in an efficient data structure. As we see an element of the stream, we determine whether it has been seen before. If it has not been seen before, then we record it in our data structure. 

If however, the number of distinct possibilities is very large, we cannot assume that this approach will work. 

## Flajolet-Martin Algorithm

We will now explore an algorithm which will allow us to estimate the number of distinct elements while utilizing a relatively small amount of memory.  

Let us suppose that elements come from some universal set $U$ and that we have some hash function which hashes the elements of $U$ to bit strings. Furthermore, let us suppose that the number of possible hashes is larger than size $U$.

The main idea behind this algorithm is that as we see more and more unique hashes, the chances increase that we will see *unusual* hashes. In this case, an *unusual* hash is something which *ends in a large number of $0$s*. 

The hash of an element $a$, $h(a)$, will end in a certain number of $0$s which we will refer to as the *tail length of $a$ and $h$*. Let $R$ denote the largest tail length so far.

Our estimate of the number of distinct elements will be
$$ 2^R $$. 

Intuitively the probability that a hash ends in at least $R$ zeros is $\frac{1}{2^R}$. If there are $m$ elements in the stream so far, then the probability that *all* of them have a length *less than $R$* is $(1-\frac{1}{2^R})^m$.

The probability that all of the elements end in fewer than $R$ zeros is
$$
(1-\frac{1}{2^R})^m \sim e^{\frac{-m}{2^R}}
$$

If $m$ is much larger than $2^R$ then this value will be close to $0$, which indicates that there is a high probablity that we will have discovered a hash ending with at least $R$ zeros. 

If $m$ is much less than $2^R$, then this probability will be close to 0. 

If $m$ and $2^R$ are comparable, then this probability will not be too high or too low and thus it is reasonable to suppose that $m$ and $2^R$ are close to one another.

Suppose we have many hash functions, $h_1, ..., h_k$. We would like to take some input, element $u$ from the universal set $U$, and we will hash that $k$ times, $h_1(u), ..., h_k(u)$, and determine a tail length for each of these, $t_1(u), t_2(u), ..., t_k(u)$. For each hash function, we will keep track of the longest tail length that we have seen so far, $R1, ...R_k$, and then we could say, take an estimate $\frac{1}{k}(2^{R_1} + ... 2^{R_k})$. We might suppose that this would give us a more stable estimate, closer to true value of unique elements, but ultimately this will not be the case.

Suppose that we have many hash functions with which we will carry out the procedure just described. It would seem reasinable that if we average the estimates, then we will get something which should be close to the true value and is more stable.

This is not possible with this method. Let us suppose that the total number of positions $N$ is large and that $m$ is the true number of unique hashes. 

## Combining Estimates

Let $p_v$ be the probability that an element hashes to a string which terminates in exactly $v$ zeros and let $q_v$ be the probability that there are fewer than $v$ zeros at the end. We can see that
$$
p_{v+1} = \frac{p_v}{2}
$$
and
$$
q_{v+1}=q_v+p_v
$$ 

Let $P_v$ denote the probability tat of all the hashes the longest terminating string of zeros is $v$. We have
$$
P_v = ∑_{j-1}^m {m \choose j}p_v^jq_v^{m-1}
$$
where $j$ is anything from $1$ to $m$ indicating the number of positions chosen.

Thus we have,

$$P_{v+1} ≥ \frac{P_v}{2}$$

*skipped the derivation*

The expected value of our estimate is
$$
∑_{v=0}^N P_v 2^v \ge (N+1)P_0 → ∞
$$
as $N→∞$.

We conclude that for some value of $m$ if we have a large number of bits being used for the hash, then we are likely to have a very large over-estimate when we average the individual estimates.

To remedy this, we could take the median, but this will always be a power of $2$.

The textbook suggests that we first group the estimates and take averages within those groups. After this, we then take the median of these averages. This will lead to a more accurate and stable estimate.